In [6]:
import torch
from meu_dataset import MeuDataset,avaliar_descritor,calcular_matching
from teste_util import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
path_dataset = "./data/datasets/features_path_flowers_dataset.pt"
# Carregar o dataset do arquivo "meu_dataset.pt"
meu_dataset = MeuDataset.load_from_file(path_dataset)
#verificar se o objeto meu dataset está retornando o tensor correto
assert isinstance(meu_dataset,MeuDataset), 'o tipo de retorno não é MeuDataset'
assert isinstance(meu_dataset[0][0],torch.Tensor), 'o tipo de retorno não é torch.Tensor'


In [7]:
import gc
from torch.optim.lr_scheduler import ExponentialLR
gc.collect()
torch.cuda.empty_cache()
batch_size_siam = 50
from torch.utils.data import random_split, DataLoader


train_dataset, val_dataset, test_dataset = random_split(meu_dataset, [0.5,0.3,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train = DataLoader(train_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_val = DataLoader(val_dataset, batch_size=batch_size_siam, shuffle=False)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size_siam, shuffle=False)

In [8]:
from tqdm import tqdm
import random

# Cosine similarity function
def cosine_similarity(a, b):
    a_norm = torch.nn.functional.normalize(a, dim=-1)
    b_norm = torch.nn.functional.normalize(b, dim=-1)
    return torch.mm(a_norm, b_norm.T)

def my_similarity(a, b):
    a_norm = torch.nn.functional.normalize(a, dim=-1)
    b_norm = torch.nn.functional.normalize(b, dim=-1)
    return torch.cdist(a_norm, b_norm, p=2)

# Triplet loss function
def triplet_loss(anchor, positive, negative, margin=0.2):
    similarities = my_similarity(anchor, positive)
    # Calcular a média da diagonal principal (âncoras vs. seus respectivos positivos)
    mean_diagonal = torch.mean(torch.diagonal(similarities))
    # Calcular a média dos outros elementos (âncoras vs. positivos não correspondentes)
    mean_other = torch.mean(similarities[~torch.eye(similarities.shape[0], dtype=torch.bool)])
    
    # losses = torch.relu(mean_other - mean_diagonal + margin)# losses considerando similaridade de cosseno
    losses = torch.relu(mean_diagonal - mean_other + margin)# losses considerando similaridade de cosseno
    # print(losses,mean_diagonal,mean_other)
    # losses = torch.relu(pos_similarity - neg_similarity + margin)
    return losses

def train_one_epoch(model, data_loader, optimizer, loss_fn, device='cpu', is_training=True):
    model.train(is_training)
    total_loss = 0.

    progress_bar = tqdm(data_loader)
    for idx, data in enumerate(progress_bar):
        # Extract the anchor and positive batches
        anchor_batch, positive_batch = (
            data[0].to(device),
            data[1].to(device),
        )

        # Calculate descriptors for the anchor and positive images
        descs_anchor = model(anchor_batch)
        descs_pos = model(positive_batch)

        # Calculate the triplet loss
        loss = loss_fn(descs_anchor, descs_pos, None, margin=3.0)

        if is_training:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item()
        progress_bar.set_description(f'Loss: {loss.item()} - Total Loss: {total_loss/len(data_loader)}')

    return total_loss/len(data_loader)


In [9]:
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
from torch import nn

class Feature(nn.Module):
    def __init__(self,n_channel=2) -> None:
        super().__init__()
        pool_size1=6
        pool_size2=3
        mult_fist_block = 4
        n_group = 12
        r2_act = gspaces.Rot2dOnR2(N=n_group)

        feat_type_in  = enn.FieldType(r2_act,  n_channel*[r2_act.trivial_repr])
        feat_type_out = enn.FieldType(r2_act, mult_fist_block*n_channel*[r2_act.regular_repr])
        self.input_type = feat_type_in

        self.block1 = enn.SequentialModule(
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=5, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                enn.GroupPooling(feat_type_out),
                )
        self.pool1 = enn.PointwiseAdaptiveAvgPool(self.block1.out_type,pool_size1)
        self.pool2 = enn.PointwiseAdaptiveAvgPool(self.block1.out_type,pool_size2)
        
        # size_after_pool = n_channel*((pool_size1**2)+(pool_size2**2))
        size_after_pool = mult_fist_block*n_channel*((pool_size1**2))
        self.dense1 = nn.Linear(size_after_pool, size_after_pool//3)
        self.dense2 = nn.Linear(size_after_pool//3, 128)
        self.dense3 = nn.Linear(size_after_pool//8, 128)
        
        self.droupout = nn.Dropout(0.2)
        self.activation = nn.LeakyReLU()
        
    def forward(self,X1)->torch.Tensor:
        x = enn.GeometricTensor(X1, self.input_type)
        n_dim = X1.shape[-1]
        mask = enn.MaskModule(self.input_type, n_dim, margin=2).to(X1.device)
        x = mask(x)
        # mais features
        x = self.block1(x)
        
        # pooling de media
        x1 = self.pool1(x)
        x2 = self.pool2(x)            
        
        # flatten
        x1 = x1.tensor.reshape(x1.shape[0],-1)
        # x2 = x2.tensor.reshape(x2.shape[0],-1)
        
        # x3 = torch.cat((x1,x2),dim=1)
        # print(x1.shape,x2.shape,x3.shape)
        # rede densa
        x3 = self.droupout(self.dense1(x1))
        x3 = self.activation(x3)
        x3 = self.droupout(self.dense2(x3))
        x3 = self.activation(x3)
        # x3 = self.droupout(self.dense3(x3))
        # x3 = self.activation(x3)
        return x3
    
n_channel =8
model =Feature(n_channel=n_channel).to(device)

In [10]:
gc.collect()
torch.cuda.empty_cache()



def train(model,dataloader_train,dataloader_val):
    optimizer_siamese = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.0001)
    scheduler = ExponentialLR(optimizer_siamese, gamma=0.75)
    best_loss = float('inf')
    best_model = None
    epochs_without_improvement = 0
    epochs = 200
    patience = 50
    
    for epoch in range(epochs):
        train_loss = train_one_epoch(model,dataloader_train , optimizer_siamese, triplet_loss, device=device, is_training=True)
        with torch.no_grad():
            test_loss = train_one_epoch(model,dataloader_val , optimizer_siamese, triplet_loss, device=device, is_training=False)

        # Verificar se a perda melhorou
        if test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
            best_model = model.state_dict()        
        else:
            epochs_without_improvement += 1
        
        if (epoch % 10 == 0) and (epoch != 0):
            scheduler.step()
            save_model(model, PATH_MODEL)
        
        # Verificar a condição de parada
        if epochs_without_improvement == patience:
            print(f"No improvement in loss for {epochs_without_improvement} epochs. Training stopped.")
            break

        print(f"Epoch [{epoch}/{epochs}] - Running Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Initial LR: {optimizer_siamese.param_groups[0]['initial_lr']:.6f}, Current LR: {optimizer_siamese.param_groups[0]['lr']:.6f}, Epochs without Improvement: {epochs_without_improvement}")

    # Carregar a melhor configuração do modelo
    model.load_state_dict(best_model)
    print(f'Epoch: {epoch}, Best Loss: {best_loss:.4f}')
    return model

PATH_MODEL = './data/models/feature_flowers_sp.pt'
model = train(model,dataloader_train,dataloader_val)

Loss: 2.491128444671631 - Total Loss: 2.4969415863355002: 100%|██████████| 60/60 [00:02<00:00, 20.89it/s] 
Loss: 2.102372407913208 - Total Loss: 2.105892797311147: 100%|██████████| 36/36 [00:00<00:00, 38.91it/s]  


Epoch [0/200] - Running Loss: 2.4969, Test Loss: 2.1059, Initial LR: 0.010000, Current LR: 0.010000, Epochs without Improvement: 0


Loss: 2.41430926322937 - Total Loss: 2.408060952027639: 100%|██████████| 60/60 [00:02<00:00, 20.30it/s]   
Loss: 2.099515676498413 - Total Loss: 2.0315516193707785: 100%|██████████| 36/36 [00:00<00:00, 37.02it/s] 


Epoch [1/200] - Running Loss: 2.4081, Test Loss: 2.0316, Initial LR: 0.010000, Current LR: 0.010000, Epochs without Improvement: 0


Loss: 2.316608190536499 - Total Loss: 2.417895205815633: 100%|██████████| 60/60 [00:02<00:00, 20.86it/s]   
Loss: 2.26814341545105 - Total Loss: 2.102235794067383: 100%|██████████| 36/36 [00:00<00:00, 38.61it/s]   


...

Loss: 2.3141286373138428 - Total Loss: 2.348312199115753: 100%|██████████| 60/60 [00:02<00:00, 20.81it/s] 
Loss: 2.0214011669158936 - Total Loss: 1.9344875514507294: 100%|██████████| 36/36 [00:00<00:00, 37.16it/s]


Epoch [12/200] - Running Loss: 2.3483, Test Loss: 1.9345, Initial LR: 0.010000, Current LR: 0.007500, Epochs without Improvement: 0


Loss: 2.3280601501464844 - Total Loss: 2.3254806756973267: 100%|██████████| 60/60 [00:02<00:00, 20.86it/s]
Loss: 1.9768012762069702 - Total Loss: 1.9375328653388553: 100%|██████████| 36/36 [00:00<00:00, 36.83it/s]


Epoch [13/200] - Running Loss: 2.3255, Test Loss: 1.9375, Initial LR: 0.010000, Current LR: 0.007500, Epochs without Improvement: 1


Loss: 2.3834986686706543 - Total Loss: 2.231308122475942:  95%|█████████▌| 57/60 [00:02<00:00, 20.52it/s] 

In [ ]:
save_model(model, PATH_MODEL)


In [ ]:
PATH_MODEL = './data/models/feature_flowers_sp.pt'
model =Feature(n_channel=8)
model.to(device)
load_model(model, PATH_MODEL,device)

model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.02)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')

In [ ]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test, model,th=0.32)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')

### Refazer o treinamento para fazer o descritor na imagem original ao inves da feature

In [ ]:
path_dataset = "./data/datasets/img_path_flowers_dataset.pt"
meu_dataset2 = MeuDataset.load_from_file(path_dataset)
train_dataset2, val_dataset2, test_dataset2 = random_split(meu_dataset2, [0.5,0.3,0.2])

# Crie uma instância do DataLoader usando seu conjunto de dados personalizado
dataloader_train2 = DataLoader(train_dataset2, batch_size=batch_size_siam, shuffle=True)
dataloader_val2 = DataLoader(val_dataset2, batch_size=batch_size_siam, shuffle=True)
dataloader_test2 = DataLoader(test_dataset2, batch_size=batch_size_siam, shuffle=True)

In [ ]:
n_channel =1
model =Feature(n_channel=n_channel).to(device)
PATH_MODEL = './data/models/img_flowers_sp.pt'
model = train(model,dataloader_train2,dataloader_val2)

In [ ]:
model =model.eval()
with torch.no_grad():
    total_acertos,total_erros,total_elementos = avaliar_descritor(dataloader_test2, model,th=0.2)
sub_conjunto = total_elementos//2
print(f'Total de elementos no DataLoader: {total_elementos}')
print(f'Acertei: {total_acertos}/{sub_conjunto} Errei: {total_erros}/{sub_conjunto}')
save_model(model, PATH_MODEL)

In [ ]:
from e2cnn import gspaces
from e2cnn import nn as enn    #the equivariant layer we need to build the model
from torch import nn
import torch

class Feature(nn.Module):
    def __init__(self,n_channel=2) -> None:
        super().__init__()
        pool_size1=9
        pool_size2=3
        n_group = 12
        r2_act = gspaces.Rot2dOnR2(N=n_group)

        feat_type_in  = enn.FieldType(r2_act,  n_channel*[r2_act.trivial_repr])
        feat_type_out = enn.FieldType(r2_act, n_channel*[r2_act.regular_repr])
        self.input_type = feat_type_in

        self.block1 = enn.SequentialModule(
                enn.R2Conv(feat_type_in, feat_type_out, kernel_size=5, padding=0, bias=False),
                enn.InnerBatchNorm(feat_type_out),
                enn.ReLU(feat_type_out, inplace=True),
                # enn.GroupPooling(feat_type_out),
                )
        self.pool1 = enn.PointwiseAdaptiveAvgPool(self.block1.out_type,pool_size1)
        self.pool2 = enn.PointwiseAdaptiveAvgPool(self.block1.out_type,pool_size2)
        
        # size_after_pool = n_channel*n_group*((pool_size1**2)+(pool_size2**2))
        size_after_pool = n_channel*n_group*((pool_size1**2))
        self.dense1 = nn.Linear(size_after_pool, size_after_pool//3)
        self.dense2 = nn.Linear(size_after_pool//3, 128)
        self.dense3 = nn.Linear(size_after_pool//8, 128)
        
        self.droupout = nn.Dropout(0.2)
        self.activation = nn.LeakyReLU()
        
    def forward(self,X1)->torch.Tensor:
        x = enn.GeometricTensor(X1, self.input_type)
        n_dim = X1.shape[-1]
        mask = enn.MaskModule(self.input_type, n_dim, margin=2).to(X1.device)
        x = mask(x)
        # mais features
        x = self.block1(x)
        
        # pooling de media
        x1 = self.pool1(x)
        x2 = self.pool2(x)            
        
        # flatten
        x1 = x1.tensor.reshape(x1.shape[0],-1)
        # x2 = x2.tensor.reshape(x2.shape[0],-1)
        
        # x3 = torch.cat((x1,x2),dim=1)
        # print(x1.shape,x2.shape,x3.shape)
        # rede densa
        x3 = self.droupout(self.dense1(x1))
        x3 = self.activation(x3)
        x3 = self.droupout(self.dense2(x3))
        x3 = self.activation(x3)
        # x3 = self.droupout(self.dense3(x3))
        # x3 = self.activation(x3)
        return x3

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_channel =8
model =Feature(n_channel=n_channel).to(device)
image_a =torch.rand(4,n_channel,32,32).to(device)
image_p =torch.rand(4,n_channel,32,32).to(device)


# Calculate descriptors for the anchor and positive images
descs_anchor = model(image_a)
descs_pos = model(image_p)

torch.ones(4, 1)
# Calculate distances/similarities between anchor and all examples in the batch
distances = cosine_similarity(descs_anchor, descs_pos)  # Broadcasting
print("distances ",distances)
# Choose the hardest negative example for each anchor
hard_negatives = torch.argmin(distances, dim=1)  # Get the index of the minimum similarity for each anchor

def similarity(desc1, desc2):
    desc1_norm = torch.nn.functional.normalize(desc1, dim=-1)
    desc2_norm = torch.nn.functional.normalize(desc2, dim=-1)
    return torch.sum(desc1_norm * desc2_norm)

print(descs_anchor.shape, descs_pos.shape)
similarity(descs_anchor, descs_pos)
